**Cross Section Spatial Models in Julia**

This is a brief tutoral detailing the use of some functions in the SpatialEconometrics.jl package. 
Questions should be directed to the e-mail: prof@alanleal-econ.com

Initially, we load some packages required to this exercise and afterwards we install the SpatialEconometrics.jl packages

1. Loading and installing required packages

In [1]:
using Pkg,DataFrames,Shapefile
Pkg.add(url="https://github.com/alanleal-econ/SpatialEconometrics.jl")
using SpatialEconometrics

    Updating git-repo `https://github.com/alanleal-econ/SpatialEconometrics.jl`
    Updating registry at `~/.julia/registries/General.toml`


Pkg.Types.PkgError: package `SpatialEconometrics [ce709044]` has same name or UUID as the active project

2. Reading and detailing data

We use data on zika cases in the Brazilian state of Ceará as available in: Amaral, P., Resende de Carvalho, L., Hernandes Rocha, T. A., da Silva, N. C., & Vissoci, J. R. N. (2019). Geospatial modeling of microcephaly and zika virus spread patterns in Brazil. PloS one, 14(9), e0222668.

ceara_zika=Shapefile.Table("/Users/alanleal/Library/CloudStorage/OneDrive-Personal/Aulas/Julia 101 uma abordagem regional/Shapefiles/ceara/ceara.shp")
W=polyneigh(ceara_zika.geometry)
W=Matrix(W) # transforma a matriz espacial em uma matrix usual
n=184 # número de observações
y = ceara_zika.inc_zik_3q # jeito de selecionar uma variável do shapefile ceara_zika
df = DataFrame(
    constant=ones(n),
    ln_gdp = ceara_zika.ln_gdp,
    ln_pop=ceara_zika.ln_pop,
    mobility = ceara_zika.mobility,
    environ = ceara_zika.environ,
    sanitation = ceara_zika.sanitation
)
X = Matrix(select(df, [:constant,:ln_gdp,:ln_pop,:mobility, :environ, :sanitation]))
X